In this notebook we will test the revision module 

In [1]:
## IMPORTs
from utils.task import Task
import json
import os
from src.pipeline.revision import revision
from utils.database_utils.db_info import get_db_schema
from dotenv import load_dotenv
from utils.prompt import load_prompt
import tiktoken
from tqdm import tqdm

load_dotenv()

True

In [2]:
# Function to load JSON data
def load_json_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data


# Function to create task object
def create_task(example):
    return Task(example)

In [3]:
# load the task data
filepath = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/subsampled_test.json"
data = load_json_data(filepath)
# load the retrieved entities
filepath_entities = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_entities.json"
retrieved_entities = load_json_data(filepath_entities)
# load the retrieved context
filepath_context = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_context.json"
retrieved_context = load_json_data(filepath_context)
# load the selected schema
filepath_query = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/generated_candidate.json"
generated_candidate = load_json_data(filepath_query)

In [4]:
# test candidate generation module with a random sample
index = 7
example = data[index]
task = create_task(example)
db_path = os.getenv("DB_ROOT_PATH") + f"/{task.db_id}/{task.db_id}.sqlite"
model = ("gpt-4")
ans = revision(task=task, retrieved_entities=retrieved_entities[index],
               retrieved_context=retrieved_context[index], generated_candidate=generated_candidate[index],
               model=model,
               num_samples=1)
ans

{'chain_of_thought_reasoning': "To solve the query, I first identified the relevant tables and columns. The 'member' table contains the member details including position, which is linked to the 'major' table through 'link_to_major' to get the college information. The position 'Secretary' is stored in the 'position' column of the 'member' table. The college information is stored in the 'college' column of the 'major' table. I joined these tables on the appropriate foreign key relationship. I ensured to filter out null values for the columns involved in the output and conditions to adhere to the database admin instructions.",
 'SQL': "SELECT T1.first_name, T1.last_name, T2.college FROM member AS T1 JOIN major AS T2 ON T1.link_to_major = T2.major_id WHERE T1.position = 'Secretary' AND T1.first_name IS NOT NULL AND T1.last_name IS NOT NULL AND T2.college IS NOT NULL"}

In [6]:
#test revision module with the subsampled test
model = "gpt-4"
num_samples = 1
res = []
for index, sample in tqdm(enumerate(data)):
    task = create_task(sample)
    ans = revision(task=task, retrieved_entities=retrieved_entities[index],
                   retrieved_context=retrieved_context[index], generated_candidate=generated_candidate[index],
                   model=model,
                   num_samples=num_samples)
    res.append(ans)
    print(str(index) + " : ", ans)

1it [00:09,  9.75s/it]

0 :  {'chain_of_thought_reasoning': "To solve the query, I first identified the tables and columns relevant to the question. The question asks for atom IDs related to a specific molecule ('TR346') and the count of distinct bond types associated with this molecule. I used the 'atom' table to fetch atom IDs and joined it with the 'bond' table to access bond types. I filtered atoms by 'molecule_id' and ensured that 'bond_type' is not null to count distinct bond types. I grouped the results by 'atom_id' to ensure each atom ID is listed with its corresponding count of bond types.", 'SQL': "SELECT T1.atom_id, COUNT(DISTINCT T2.bond_type) AS bond_type_count FROM atom AS T1 JOIN bond AS T2 ON T1.molecule_id = T2.molecule_id WHERE T1.molecule_id = 'TR346' AND T2.bond_type IS NOT NULL GROUP BY T1.atom_id"}


2it [00:24, 12.47s/it]

1 :  {'chain_of_thought_reasoning': "To find the city location of the high school with specific criteria, I need to join the 'schools' and 'frpm' tables because the 'NSLP Provision Status' information is in the 'frpm' table and the school level and city information is in the 'schools' table. The conditions specified are that the school must be a high school ('EILCode' = 'HS'), located in Merced County, with 'NSLP Provision Status' as 'Lunch Provision 2', and the grade span served must be from 9 to 12. I will use an INNER JOIN to combine these tables on the 'CDSCode' field, which is a common key. The query will filter based on these conditions and select the 'City' from the 'schools' table.", 'SQL': "SELECT T1.City FROM schools AS T1 INNER JOIN frpm AS T2 ON T1.CDSCode = T2.CDSCode WHERE T1.EILCode = 'HS' AND T1.County = 'Merced' AND T2.`NSLP Provision Status` = 'Lunch Provision 2' AND T1.GSserved = '9-12'"}


3it [00:32, 10.44s/it]

...

12 :  {'chain_of_thought_reasoning': "The query aims to calculate the percentage of bonds that are double bonds (' = ') for the molecule with ID 'TR047'. The calculation is done by dividing the number of double bonds by the total number of bonds for this molecule, then multiplying by 100 to get the percentage. The WHERE clause filters bonds belonging to 'TR047' and ensures that bond_type is not null to avoid incorrect calculations. The table name 'bonds' was incorrect in the initial query and has been corrected to 'bond'.", 'SQL': "SELECT (SUM(CASE WHEN bond_type = ' = ' THEN 1 ELSE 0 END) * 100.0 / COUNT(bond_id)) AS percent FROM bond T1 WHERE T1.molecule_id = 'TR047' AND T1.bond_type IS NOT NULL"}


14it [02:36, 11.68s/it]

13 :  {'chain_of_thought_reasoning': "To solve the query, I first identified the relevant tables: 'votes' for bounty information, 'posts' for post titles, and 'users' for user display names. I then joined these tables based on their relationships: 'votes' with 'posts' on PostId, and 'votes' with 'users' on UserId. The WHERE clause filters for a BountyAmount of 50 and titles containing 'variance'. The SELECT clause only includes the DisplayName from the 'users' table, as per the instructions.", 'SQL': "SELECT T3.DisplayName FROM votes T1 JOIN posts T2 ON T1.PostId = T2.Id JOIN users T3 ON T1.UserId = T3.Id WHERE T1.BountyAmount = 50 AND T2.Title LIKE '%variance%'"}


15it [02:45, 11.02s/it]

14 :  {'chain_of_thought_reasoning': "The initial query attempted to select a non-existent column 'DateObtained' from the 'Badges' table. The correct column name, as per the schema, is 'Date'. The table aliases were also incorrectly referenced. The 'Users' table should be aliased as T1 and the 'Badges' table as T2. The join condition should correctly reference 'T1.Id' and 'T2.UserId' based on the foreign key relationship defined in the schema. The WHERE clause correctly filters users based on their location in 'Rochester, NY'.", 'SQL': "SELECT T2.Date FROM Users T1 JOIN Badges T2 ON T1.Id = T2.UserId WHERE T1.Location = 'Rochester, NY'"}


In [7]:
from pathlib import Path

schema_path = Path('C:/Users/yousf/Bureau/ConvergenceAI/CHESS_Impl/data/test/final_sql.json')
with open(schema_path, 'w') as f:
    json.dump(res, f, indent=4)